In [15]:
import hazm
import nltk
import pandas
import re 
import pickle
import arabic_reshaper

In [3]:
df = pandas.read_excel('final_books.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     2824 non-null   object
 1   date      2824 non-null   object
 2   content   2441 non-null   object
 3   category  2824 non-null   object
 4   author    2824 non-null   object
 5   comments  2824 non-null   object
dtypes: object(6)
memory usage: 132.5+ KB


In [4]:
df.iloc[502]['content']

'\xa0\xa0مترجم در پیشگفتار این گونه بیان می کند که کتاب “سلسله های اسلامی” که از نظر خوانندگان می گذرد، می تواند یک کتاب مراجعه کوچک در زمینه تاریخ اسلام به شمار آید، و مراد من از کتاب مراجعه، آن چنان کتابی است که فرنگی ها بدان کتاب رفرنس می گویندو تفاوت چنین کتابی با کتابهای معمولی که درباره تاریخ کشورهای اسلامی نوشته شده است در این است که این کتاب، و اصولا همه کتابهای مراجعه، برای آن نیست که خواننده از آغاز تا انجام آن یکباره مطالعه کند، بلکه برای آن است که برای رفع احتیاجات فوری خود بدان مراجعه کند، و جواب پرسش یا مشکل خود را در آن بیابد، و به این ترتیب، با صرف مقدار ناچیزی از وقت به آنچه می خواهد برسد.,مترجم در پیشگفتار این گونه بیان می کند که کتاب “سلسله های اسلامی” که از نظر خوانندگان می گذرد، می تواند یک کتاب مراجعه کوچک در زمینه تاریخ اسلام به شمار آید، و مراد من از کتاب مراجعه، آن چنان کتابی است که فرنگی ها بدان کتاب رفرنس می گویند و تفاوت چنین کتابی با کتابهای معمولی که درباره تاریخ کشورهای اسلامی نوشته شده است در این است که این کتاب، و اصولا همه کتابهای مراجعه، برای آن نیست 

In [5]:
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~،'''
    for char in text:
        if char in punctuations:
            text = text.replace(char, "")
    return text

In [7]:
import hazm

normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
stop_words = hazm.stopwords_list()

tokens = []
lemmatized_tokens = []
for index, row in df.iterrows():
    content = row['content'] 
    norm_content = normalizer.normalize(str(content))
    norm_content_no_punctuations = remove_punctuations(norm_content)
    tokenized_sentences = hazm.sent_tokenize(norm_content_no_punctuations)
    content_tokenized_words = []
    for sentence in tokenized_sentences:
        try:
            tokenized_words = hazm.word_tokenize(sentence)
            filtered_words = [word for word in tokenized_words if word not in stop_words]
            content_tokenized_words.extend(filtered_words)
            for word in filtered_words:
                lemmatizedWord = lemmatizer.lemmatize(word)
                lemmatized_tokens.append((index, lemmatizedWord))
        except:
            print("error")  
    tokens.extend(content_tokenized_words)

In [8]:
def create_word_index(arr):
    word_index = {}
    for doc_id, word in arr:
        if word in word_index:
            word_index[word].append(doc_id)
        else:
            word_index[word] = [doc_id]
    return dict(sorted(word_index.items(), key=lambda x: len(x[1]), reverse=True))

In [9]:
word_index = create_word_index(lemmatized_tokens)

In [10]:
first_key, first_value = next(iter(word_index.items()))
print(first_key, first_value[0:100])

کتاب [0, 1, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 11, 13, 13, 13, 13, 13, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 18, 18, 19, 21, 21, 21, 21, 22, 24, 25, 25, 26, 27, 27, 27, 29, 30, 30, 30, 34, 35, 37, 37, 37, 38, 39, 40, 41, 43, 43, 43, 44, 44, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 48, 48, 49, 50]


In [11]:
def sort_dict_values(d):
    sorted_dict = {}
    for k in d:
        sorted_dict[k] = sorted(set(d[k]), key=d[k].count, reverse=True)
    return sorted_dict

In [12]:
sorted_word_index = sort_dict_values(word_index)
first_key, first_value = next(iter(sorted_word_index.items()))
print(first_key, first_value[0:10])

کتاب [502, 512, 428, 960, 1583, 103, 112, 188, 544, 8]


In [16]:
# Save the word_dictionaries object to a file
with open('index_dict.pkl', 'wb') as f:
    pickle.dump(sorted_word_index, f)